In [ ]:
import openl3
import soundfile as sf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio
from multiprocessing import Pool
import random
import pickle 
from pytube import YouTube
from moviepy.editor import *
from scipy.spatial import distance
import os, glob
from pydub import AudioSegment, effects
import soundfile as sf
HOP_SIZE = 0.2 # when embeddings sounds, shift windows by HOP_SIZE
GLOBAL_SR = 44100
CONTENT_TYPE = "music"
INPUT_REPR = "mel128"

In [ ]:
def make_mono(stereo_sound):
    # given a stereo sound, return mono 
    mono_sound = stereo_sound[0,:]
    for s in stereo_sound[1:]:
        mono_sound += s
    return mono_sound/len(stereo_sound)

def download_youtube_audio(url,name=False):
    yt = YouTube(url) # download audio from youtube link with p
    stream = yt.streams.filter(only_audio=True).first()
    if name:
        path = stream.download("media/", filename = name)
    else:
        path = stream.download("media/")
    return AudioFileClip(path) # load the audio using moviepy.editor

# bag a sound, and save it
def bag_n_save(sound,name,overwrite=False):
    if os.path.isfile(name)==False or overwrite==True:
        pass
    else:
        return True
    embeddings, ts = openl3.get_audio_embedding(sound,sr=GLOBAL_SR,frontend='librosa',input_repr=INPUT_REPR,hop_size=0.2,
                                     embedding_size=512,content_type=CONTENT_TYPE,verbose=1,)
    bag = list(zip(embeddings,sound_segments))
    with open(name,"wb") as f:
        pickle.dump(bag,f)
    return bag

def make_bags():
    # get evey media file, bag it and save it in the bags folder
    medias = glob.glob("media/*.wav")
    for mpath in medias:
        audio, sr = sf.read(mpath)
        bag_name = os.path.basename(mpath).split(".")[0]
        bag_n_save(sound,"media/%s.bag"%bag_name)


In [ ]:
# download the songs
download_youtube_audio('https://www.youtube.com/watch?v=wp43OdtAAkM',"katebush_runningup.mp4")
download_youtube_audio('https://www.youtube.com/watch?v=m1cRGVaJF7Y','front242_headhunter.mp4')

In [ ]:
# convert songs to wav after normalizing mp4s
name = "front242_headhunter"
s = AudioFileClip("media/%s.mp4"%name,).fx(afx.audio_normalize)
s.write_audiofile("media/%s.wav"%name,codec='pcm_s16le')
name = "katebush_runningup"
s = AudioFileClip("media/%s.mp4"%name,).fx(afx.audio_normalize)
s.write_audiofile("media/%s.wav"%name,codec='pcm_s16le')

In [ ]:
# embed both songs
audio_kb, sr = sf.read('media/katebush_runningup.wav')
assert(sr==GLOBAL_SR)
kb_emb, kb_times = openl3.get_audio_embedding(audio_kb,sr=GLOBAL_SR,frontend='librosa',input_repr=INPUT_REPR,hop_size=HOP_SIZE,
                                     embedding_size=512,content_type=CONTENT_TYPE,verbose=1)
kb = dict(zip(kb_times,kb_emb))

audio_f242, sr = sf.read('media/front242_headhunter.wav')
assert(sr==GLOBAL_SR)
f242_emb, f242_times = openl3.get_audio_embedding(audio_f242,sr=GLOBAL_SR,frontend='librosa',input_repr=INPUT_REPR,hop_size=HOP_SIZE,
                                     embedding_size=512,content_type=CONTENT_TYPE,verbose=1)
f242 = dict(zip(f242_times,f242_emb))


In [ ]:
def closest_node(node, nodes,metric='euclidean'):
    ''' distance measuring function''' 
    closest_index = distance.cdist([node], nodes,metric = metric).argmin()
    return closest_index,nodes[closest_index]

remade = []
for i in range(int(len(kb.keys())*HOP_SIZE)):
    t,_ = closest_node(kb[i],f242_emb,metric="cosine")
    start_time = int(0.2*t*GLOBAL_SR)
    end_time = start_time+GLOBAL_SR
    remade.append(audio_f242[start_time:end_time,:])
Audio(np.vstack(remade).T,rate=GLOBAL_SR)

In [ ]:
remade = []
for i in range(int(len(f242.keys())*HOP_SIZE)):
    t,_ = closest_node(f242[i],kb_emb,metric="euclidean")
    start_time = int(0.2*t*GLOBAL_SR)
    end_time = start_time+GLOBAL_SR
    remade.append(audio_kb[start_time:end_time,:])
Audio(np.vstack(remade).T,rate=GLOBAL_SR)

In [ ]:
audio_kb, sr = sf.read('media/katebush_runningup.wav')
Audio(audio_kb.T,rate=sr)

In [ ]:
audio_kb, sr = sf.read('media/front242_headhunter.wav')
Audio(audio_kb.T,rate=sr)